In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [24]:
import sys
import os
import pandas as pd
import numpy as np

In [25]:
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
# Lấy thư mục hiện tại
current_directory = os.getcwd()
folder_path1 = os.path.join(current_directory,'../../data/train_test')

In [26]:
x = pd.read_csv(f'{folder_path1}/xtrain_data.csv')
y = pd.read_csv(f'{folder_path1}/xtest_data.csv')
z = pd.read_csv(f'{folder_path1}/ytrain_data.csv')
t = pd.read_csv(f'{folder_path1}/ytest_data.csv')

x_train = x.iloc[:, 1:].values
x_test = y.iloc[:, 1:].values

y_train = z['drop_out'].values
y_test = t['drop_out'].values


In [27]:
rf =  RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

RandomForestRegressor(random_state=42)

In [28]:
y_pred = rf.predict(x_test)

In [29]:
# Chuyển đổi y_train thành nhãn phân loại.
y_test_class = np.where(y_test >= 0.5, 1, 0)
y_pred_class = np.where(y_pred >= 0.5, 1, 0)
y_train_class = np.where(y_train >= 0.5, 1, 0)

In [30]:
# Đánh giá mô hình
accuracy = accuracy_score(y_test_class , y_pred_class)
conf_matrix = confusion_matrix(y_test_class , y_pred_class)
class_report = classification_report(y_test_class, y_pred_class)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(class_report)


Accuracy: 1.0
Confusion Matrix:
[[64  0]
 [ 0 63]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        64
           1       1.00      1.00      1.00        63

    accuracy                           1.00       127
   macro avg       1.00      1.00      1.00       127
weighted avg       1.00      1.00      1.00       127



In [31]:
from sklearn.dummy import DummyClassifier # mô hình cơ sỏ
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(x_train, y_train)
baseline_prediction = dummy_clf.predict(x_test)
baseline_accuracy = accuracy_score(y_test_class, np.where(baseline_prediction==np.unique(baseline_prediction)[0],0,baseline_prediction))

In [32]:
print("Model accuracy:", accuracy)
print("Baseline accuracy:", baseline_accuracy)

Model accuracy: 1.0
Baseline accuracy: 0.5039370078740157


In [33]:
[[64  0]
 [ 0 63]]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3318387125.py, line 1)

In [34]:
# Độ chính xác toàn thể: (TN+TP)/N
print("Độ chính xác toàn thể: ", round(accuracy*100,2))
# Sai số toàn thể
overall_mistake = round((0+0)/x_test.shape[0] * 100,2)
print("Sai số toàn thể: ", overall_mistake)

# Độ nhạy - % dự đoán chính xác sinh viên thôi học: TP/(TP+FN)
sensitivity = (1/(1+0)) * 100
print("Độ nhạy: ", sensitivity)

# Sai số của dự đoán sinh viên thôi TP/(TP+FN)
mistake = 100 - sensitivity
print("Tỉ lệ mô hình dự đoán sai sinh viên nghỉ học: ", mistake)
 
# Độ đặc hiệu (specificity), % dự đoán sinh viên không thôi học: TN/(TN+FP)

specificity = (64/(64+0))*100
print("Độ đặc hiệu: ", specificity)
mistake_specificity = 100 - specificity
print("Sai số tỉ lệ mô hình dự đoán chính xác sinh viên không thôi học: ", mistake_specificity)

Độ chính xác toàn thể:  100.0
Sai số toàn thể:  0.0
Độ nhạy:  100.0
Tỉ lệ mô hình dự đoán sai sinh viên nghỉ học:  0.0
Độ đặc hiệu:  100.0
Sai số tỉ lệ mô hình dự đoán chính xác sinh viên không thôi học:  0.0


In [ ]:
student_id = y.iloc[:, 1]
df_pred = pd.concat([student_id,pd.DataFrame(y_test_class)], axis = 1).rename(columns = {0: 'drop_out'})
df_pred.rename(columns = {0: 'drop_out'}, inplace=True)

In [ ]:
# Kết quả dự đoán
df_pred[df_pred['drop_out']==1]

,student_id,drop_out
0,2011956334,1
1,2021595788,1
3,1954177532,1
4,2000001302,1
6,1983739846,1
...,...,...
116,2200003944,1
118,2000004424,1
121,2037245042,1
122,2114305523,1


In [ ]:
from statsmodels.stats.proportion import proportion_confint # Tính khoảng tin cậy
lower_bound, upper_bound = proportion_confint(sum(y_pred_class), len(y_pred_class), alpha=0.05, method='wilson')
print('lower_bound: ', lower_bound) # Giới hạn của khoảng tin cậy dưới
print('upper_bound: ', upper_bound) # Giới hạn của khoàng tin cậy trên

lower_bound:  0.4105078767844485
upper_bound:  0.5818492857769197


In [ ]:
class_report = classification_report(y_test_class, y_pred_class, output_dict=True)
df_class_report_rdf = pd.DataFrame(class_report)
df_class_report_rdf.to_csv('../visualization/data/randomforest.csv')